In [1]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [651]:
import pandas as pd

url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
shark_df = pd.read_excel(url)

In [652]:
shark_df = shark_df.drop(['pdf', 'href formula','href','Case Number','Case Number.1','original order','Unnamed: 21','Unnamed: 22','Unnamed: 11','Source'], axis=1)

In [653]:
shark_df = shark_df.dropna(how='all')

In [654]:
shark_df = shark_df.dropna(subset=['Country', 'State', 'Location'])

In [655]:
shark_df.columns = shark_df.columns.str.lower()

In [656]:
shark_df.columns

Index(['date', 'year', 'type', 'country', 'state', 'location', 'activity',
       'name', 'sex', 'age', 'injury', 'time', 'species '],
      dtype='object')

# 1. Clean cloumn Year

In [657]:
# Check how many empty cells are in column year
shark_df['year'].isna().sum()

1

In [658]:
# Only one empty cell in cloumn year, so we can replace with the Mode of column year
shark_df.year.fillna(shark_df.year.mode()[0], inplace=True)

In [659]:
# See if in column year, are any irregularities
shark_df['year'].unique()

array([2024., 2023., 2022., 2021., 2020., 2019., 2018., 2017., 2016.,
       2015., 2014., 2013., 2012., 2011., 2010., 2009., 2008., 2007.,
       2006., 2005., 2004., 2003., 2002., 2001., 2000., 1999., 1998.,
       1997., 1996., 1995., 1984., 1994., 1993., 1992., 1991., 1990.,
       1989., 1988., 1987., 1986., 1985., 1983., 1982., 1981., 1980.,
       1979., 1978., 1977., 1976., 1975., 1974., 1973., 1972., 1971.,
       1970., 1969., 1968., 1967., 1966., 1965., 1964., 1963., 1962.,
       1961., 1960., 1959., 1958., 1957., 1956., 1955., 1954., 1953.,
       1952., 1951., 1950., 1949., 1948., 1848., 1947., 1946., 1945.,
       1944., 1943., 1942., 1941., 1940., 1939., 1938., 1937., 1936.,
       1935., 1934., 1933., 1932., 1931., 1930., 1929., 1928., 1927.,
       1926., 1925., 1924., 1923., 1922., 1921., 1920., 1919., 1918.,
       1917., 1916., 1915., 1914., 1913., 1912., 1911., 1910., 1909.,
       1908., 1907., 1906., 1905., 1904., 1903., 1902., 1901., 1900.,
       1899., 1898.,

In [660]:
# Change all strings to integers
shark_df['year'] = shark_df['year'].astype(int)

In [661]:
# Filter DataFrame for years with less than 4 chracters
condition = shark_df['year'].astype(str).apply(len) < 4
filtered_df = shark_df[condition]
filtered_df

,date,year,type,country,state,location,activity,name,sex,age,injury,time,species
6802,Ca. 5 A.D.,5,Unprovoked,AUSTRALIA,New South Wales,Bondi,NaN,male,M,NaN,Aboriginal rock carving depicts man being atta...,NaN,NaN
6804,Ca. 336.B.C..,0,Unprovoked,GREECE,Piraeus,In the haven of Cantharus,Washing his pig in preparation for a religious...,A candidate for initiation,M,NaN,"FATAL, shark ""bit off all lower parts of him u...",NaN,NaN
6806,Ca. 725 B.C.,0,Sea Disaster,ITALY,Tyrrhenian Sea,Krater found during excavations at Lacco Ameno...,Shipwreck,males,M,NaN,Depicts shipwrecked sailors attacked by a sha...,NaN,NaN
6808,Ca 4000 BC,0,NaN,PERU,Paloma,Archeological site,NaN,male,M,17,FATAL,NaN,NaN
6810,After 2013,0,Unprovoked,AUSTRALIA,Queensland,Otter Reef,Spearfishing,Reece Pla,M,NaN,"Shark bumped him, but no injury",NaN,Hammerhead shark
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922,Circa 1862,0,Unprovoked,USA,Hawaii,Puna,NaN,"A ""chiefess""",F,NaN,Ankle bitten,NaN,NaN
6926,Before 1906,0,Unprovoked,AUSTRALIA,New South Wales,,Swimming,Arab boy,M,NaN,FATAL,NaN,Said to involve a grey nurse shark that leapt ...
6927,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,NaN,NaN
6929,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,NaN,NaN


In [662]:
# Get the unique value of these cells
result = shark_df['year'].unique()
smaller_than_4 = [value for value in result if len(str(value)) < 4]
print(smaller_than_4)

[5, 0]


In [663]:
# Replace these values with the Mode of the column year
for item in smaller_than_4:
    shark_df['year'] = shark_df['year'].replace(item, shark_df.year.mode()[0])

In [664]:
# Check again for irregularities
shark_df['year'].unique()

array([2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014,
       2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003,
       2002, 2001, 2000, 1999, 1998, 1997, 1996, 1995, 1984, 1994, 1993,
       1992, 1991, 1990, 1989, 1988, 1987, 1986, 1985, 1983, 1982, 1981,
       1980, 1979, 1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970,
       1969, 1968, 1967, 1966, 1965, 1964, 1963, 1962, 1961, 1960, 1959,
       1958, 1957, 1956, 1955, 1954, 1953, 1952, 1951, 1950, 1949, 1948,
       1848, 1947, 1946, 1945, 1944, 1943, 1942, 1941, 1940, 1939, 1938,
       1937, 1936, 1935, 1934, 1933, 1932, 1931, 1930, 1929, 1928, 1927,
       1926, 1925, 1924, 1923, 1922, 1921, 1920, 1919, 1918, 1917, 1916,
       1915, 1914, 1913, 1912, 1911, 1910, 1909, 1908, 1907, 1906, 1905,
       1904, 1903, 1902, 1901, 1900, 1899, 1898, 1897, 1896, 1895, 1894,
       1893, 1892, 1891, 1890, 1889, 1888, 1887, 1886, 1885, 1884, 1883,
       1882, 1881, 1880, 1879, 1878, 1877, 1876, 18

# 2. Clean column Date

In [665]:
# Check how many empty cells are in column date
shark_df['date'].isna().sum()

0

In [666]:
# See if in column date, are any irregularities
shark_df['date'].unique()

array(['09-Jan-2024', '05-Jan-2024', '30 Dec-2023', ..., 'Before 1903',
       '1900-1905', '1845-1853'], dtype=object)

In [667]:
# format
shark_df["date"] = shark_df["date"].str.replace('  ', '-').str.replace(' ', '-').str.replace('--', '-').str.replace('Reported', '').str.strip()
shark_df["date"] = shark_df["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: '-'.join(filter(lambda x: x and len(x) <= 10, date_list)))
shark_df["date"] = shark_df["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: date_list[:2] + [date_list[2][:3]] if len(date_list) == 3 else date_list).apply(lambda date_list: '-'.join(date_list))
shark_df["date"] = shark_df["date"].apply(lambda x: x.split('-') if pd.notna(x) else []).apply(lambda date_list: ['01', 'Jan'] + date_list if len(date_list) == 1 and len(date_list[0]) == 4 else date_list).apply(lambda date_list: '-'.join(date_list) if date_list else '')


In [668]:
shark_df['date'].unique()

array(['09-Jan-202', '05-Jan-202', '30-Dec-202', ..., 'Before-1903',
       '1900-1905', '1845-1853'], dtype=object)

In [669]:
#shark_df["date"] = pd.to_datetime(shark_df.date,format='%d-%m-%y')

In [670]:
# date_formats = ['%d-%m-%y', '%d-%b-%Y']

# for format in date_formats:
#     try:
#         shark_df["date"] = pd.to_datetime(shark_df["date"], format=format)
#         break  # Break out of the loop if a format is successfully applied
#     except ValueError:
#         print(f"Unable to parse dates with format {format}")
#         print("Problematic dates:")
#         print(shark_df.loc[pd.to_datetime(shark_df["date"], errors='coerce').isnull(), "date"])
#         shark_df["date"] = ''  # Set an empty string if the format is not successfully applied

# # Now, shark_df["date"] should contain the parsed dates with the correct format, or empty strings for unsuccessful cases.

In [671]:
format_1 = '%d-%m-%y'
format_2 = '%d-%b-%Y'
format_3 = '%d-%B-%Y'

try:
    shark_df["date"] = pd.to_datetime(shark_df["date"], format=format_1)
except ValueError:
    try:
        shark_df["date"] = pd.to_datetime(shark_df["date"], format=format_2)
    except ValueError:
        try:
            shark_df["date"] = pd.to_datetime(shark_df["date"], format=format_3)
        except ValueError:
            print(f"Unable to parse dates with formats {format_1}, {format_2}, and {format_3}")
            print("Problematic dates:")
            problematic_dates = shark_df.loc[pd.to_datetime(shark_df["date"], errors='coerce').isnull(), "date"]
            print(problematic_dates)
            shark_df["date"] = ''  # Set an empty string if the formats are not successfully applied


Unable to parse dates with formats %d-%m-%y, %d-%b-%Y, and %d-%B-%Y
Problematic dates:
0        09-Jan-202
1        05-Jan-202
2        30-Dec-202
3        29-Dec-202
4        28-Dec-202
           ...     
6922     Circa-1862
6926    Before-1906
6927    Before-1903
6929      1900-1905
6931      1845-1853
Name: date, Length: 5779, dtype: object


C:\Users\vonke\AppData\Local\Temp\ipykernel_8164\199533571.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  problematic_dates = shark_df.loc[pd.to_datetime(shark_df["date"], errors='coerce').isnull(), "date"]


In [672]:
shark_df["date"]

0        
1        
2        
3        
4        
       ..
6922     
6926     
6927     
6929     
6931     
Name: date, Length: 6132, dtype: object

In [673]:
shark_df['date'].isna().sum()

0

In [674]:
# Filter DataFrame  with less than 4 chracters
condition = shark_df['date']
filtered_df = shark_df[condition]
filtered_df

KeyError: "None of [Index(['', '', '', '', '', '', '', '', '', '',\n       ...\n       '', '', '', '', '', '', '', '', '', ''],\n      dtype='object', length=6132)] are in the [columns]"